# 💤 [Day 23](https://adventofcode.com/2019/day/23)

In [77]:
def run_program(p, inputs, init_op=0, init_base=0):
    """Intcode as usual"""
    #Inputs are given in reverse order (pop)
    op = init_op
    relative_base = init_base
    last_diagnostic = []
    while p[op] != 99:
        codes = "%05d" % p[op]
        codes = [int(codes[0]), int(codes[1]), int(codes[2]), int(codes[3:])]
        # inputs
        if codes[-1] == 3:
            if not len(inputs):
                # Game is waiting for next move
                break
            assert codes[1] == 0
            p[p[op + 1] + (relative_base if codes[2] == 2 else 0)] = inputs.pop()
            op += 2
        # unary ops
        elif codes[-1] in  [4, 9]:
            # read parameter
            assert codes[1] == 0
            param = p[op + 1]
            if (codes[2] % 2) == 0:
                try:
                    param = p[param + (relative_base if codes[2] == 2 else 0)]
                except IndexError:
                    param = 0
            # output
            if codes[-1] == 4:
                last_diagnostic.append(param)
            # update relative base
            else:
                relative_base += param
            # next instr
            op += 2
        else:
            # read parameters in correct mode
            x, y = p[op + 1:op + 3]
            if (codes[2] % 2) == 0:
                try:
                    x = p[x + (relative_base if codes[2] == 2 else 0)]
                except IndexError:
                    x = 0
            if not (codes[1] % 2):
                try:
                    y = p[y + (relative_base if codes[1] == 2 else 0)]
                except IndexError:
                    y = 0
            # Read target and allocate more memory if needed
            target = p[op + 3] + (relative_base if codes[0] == 2 else 0)
            if target >= len(p): 
                p += [0] * (target - len(p) + 1)
            # addition and multiplication
            if codes[-1] in [1, 2]:
                p[target] = x + y if codes[-1] == 1 else x * y
                op += 4
            # Comparison result
            elif codes[-1] == 7:
                p[target] = int(x < y)
                op += 4
            elif codes[-1] == 8:
                p[target] = int(x == y)
                op += 4
            # Jump if eq
            elif (codes[-1] == 5 and x != 0) or (codes[-1] == 6 and x == 0):
                op = y  
            # Jump instruction that failed their test
            else:
                op += 3
    return last_diagnostic, 0, op, relative_base


from collections import defaultdict
def boot(program, n=50, part1=True):
    computers = [[[x for x in program], 0, 0] for _ in range(n)]
    pkt_queue = defaultdict(lambda: [])
    idle = {i: False for i in range(n)}
    last_nat_y = None
    timestamp = 0
    
    while 1:
        # If everyone is idle, send last packet to NAT and resumes
        if sum(idle.values()) == n:
            pkt_queue[0] = pkt_queue[255]
            print("[t={}] NAT sends packet X={}, Y={}".format(
                timestamp, pkt_queue[255][0][0], pkt_queue[255][0][1]))
            # monitor Y values
            if last_nat_y == pkt_queue[255][0][1]:
                return pkt_queue[255][0][1]
            last_nat_y = pkt_queue[255][0][1]
            # resume activity
            for i in range(n):
                idle[i] = True
                
        # Execute computers
        for i in range(n):
            # Determine inputs
            if timestamp == 0:
                inputs = [-1, i]
            elif len(pkt_queue[i]) == 0:
                inputs = [-1]
                idle[i] = True
            else:
                inputs = sorted(pkt_queue[i], key=lambda x: (x[2], x[3]))
                inputs = [z for x, y, _, _ in inputs for z in [x, y]][::-1]
                pkt_queue[i] = []
                
            # Run computers
            assert len(inputs) >= 1
            out, _, computers[i][1], computers[i][2] = run_program(
                computers[i][0], 
                inputs=inputs, 
                init_op=computers[i][1], 
                init_base=computers[i][2])
            
            # Idle ?
            idle[i] = idle[i] and (not len(out))
            
            # Add packets to queue
            for z in range(len(out) // 3):
                tgt, X, Y = out[3  * z:3  *z  +3]
                pkt_queue[tgt].append((X, Y, timestamp, z))
                
        if 255 in pkt_queue:
            # Part1, detect first packet sent to nat
            if part1:
                return timestamp, min(pkt_queue[255], key=lambda x: (x[2], x[3]))[1]
            # Part2, only keep latest packet sent to NAT
            else:
                pkt_queue[255] = [max(pkt_queue[255], key=lambda x: (x[2], x[3]))]
        timestamp += 1

In [78]:
with open("inputs/day23.txt", 'r') as f:
    inputs = list(map(int, f.read().split(',')))
    
print("The first packet sent to adress 255 occurs at timestamp {} with Y={}".format(
    *boot(inputs, part1=True)))

The first packet sent to adress 255 occurs at timestamp 5 with Y=15416


In [79]:
print("\nThe first packet to be sent twice in a row to NAT is Y={}".format(boot(inputs, part1=False)))

[t=7] NAT sends packet X=102877, Y=15416
[t=9] NAT sends packet X=102877, Y=14164
[t=11] NAT sends packet X=102877, Y=13231
[t=13] NAT sends packet X=102877, Y=12557
[t=15] NAT sends packet X=102877, Y=12077
[t=17] NAT sends packet X=102877, Y=11739
[t=19] NAT sends packet X=102877, Y=11501
[t=21] NAT sends packet X=102877, Y=11334
[t=23] NAT sends packet X=102877, Y=11217
[t=25] NAT sends packet X=102877, Y=11135
[t=27] NAT sends packet X=102877, Y=11078
[t=29] NAT sends packet X=102877, Y=11038
[t=31] NAT sends packet X=102877, Y=11010
[t=33] NAT sends packet X=102877, Y=10990
[t=35] NAT sends packet X=102877, Y=10976
[t=37] NAT sends packet X=102877, Y=10967
[t=39] NAT sends packet X=102877, Y=10960
[t=41] NAT sends packet X=102877, Y=10955
[t=43] NAT sends packet X=102877, Y=10952
[t=45] NAT sends packet X=102877, Y=10950
[t=47] NAT sends packet X=102877, Y=10948
[t=49] NAT sends packet X=102877, Y=10947
[t=51] NAT sends packet X=102877, Y=10946
[t=53] NAT sends packet X=102877, Y=